In [1]:
from pyamaze import maze, agent, COLOR, textLabel
from timeit import timeit
from queue import PriorityQueue
from math import sqrt

In [2]:
# heuristic function as Euclidean Distance
def h2(cell1, cell2):
    x1, y1 = cell1
    x2, y2 = cell2
    return sqrt((x1-x2)**2+(y1-y2)**2)
    # return ((x1-x2)**2+(y1-y2)**2)

# heuristic function as Manhattan Distance
def h1(cell1, cell2):
    x1, y1 = cell1
    x2, y2 = cell2
    return (abs(x1 - x2) + abs(y1 - y2))


In [3]:
def aStar1(m, start=None):
    if start is None:
        start = (m.rows, m.cols)
    open = PriorityQueue()
    open.put((h1(start, m._goal), h1(start, m._goal), start))
    aPath = {}
    g_score = {row: float("inf") for row in m.grid}
    g_score[start] = 0
    f_score = {row: float("inf") for row in m.grid}
    f_score[start] = h1(start, m._goal)
    searchPath = [start]
    while not open.empty():
        currCell = open.get()[2]
        searchPath.append(currCell)
        if currCell == m._goal:
            break
        for d in 'ESNW':
            if m.maze_map[currCell][d] == True:
                if d == 'E':
                    childCell = (currCell[0], currCell[1]+1)
                elif d == 'W':
                    childCell = (currCell[0], currCell[1]-1)
                elif d == 'N':
                    childCell = (currCell[0]-1, currCell[1])
                elif d == 'S':
                    childCell = (currCell[0]+1, currCell[1])

                temp_g_score = g_score[currCell] + 1
                temp_f_score = temp_g_score + h1(childCell, m._goal)

                if temp_f_score < f_score[childCell]:
                    aPath[childCell] = currCell
                    g_score[childCell] = temp_g_score
                    f_score[childCell] = temp_g_score + h1(childCell, m._goal)
                    open.put((f_score[childCell], h1(
                        childCell, m._goal), childCell))

    fwdPath = {}
    cell = m._goal
    while cell != start:
        fwdPath[aPath[cell]] = cell
        cell = aPath[cell]
    return searchPath, aPath, fwdPath

In [4]:
def aStar2(m, start=None):
    if start is None:
        start = (m.rows, m.cols)
    open = PriorityQueue()
    open.put((h2(start, m._goal), h2(start, m._goal), start))
    aPath = {}
    g_score = {row: float("inf") for row in m.grid}
    g_score[start] = 0
    f_score = {row: float("inf") for row in m.grid}
    f_score[start] = h2(start, m._goal)
    searchPath = [start]
    while not open.empty():
        currCell = open.get()[2]
        searchPath.append(currCell)
        if currCell == m._goal:
            break
        for d in 'ESNW':
            if m.maze_map[currCell][d] == True:
                if d == 'E':
                    childCell = (currCell[0], currCell[1]+1)
                elif d == 'W':
                    childCell = (currCell[0], currCell[1]-1)
                elif d == 'N':
                    childCell = (currCell[0]-1, currCell[1])
                elif d == 'S':
                    childCell = (currCell[0]+1, currCell[1])

                temp_g_score = g_score[currCell] + 1
                temp_f_score = temp_g_score + h2(childCell, m._goal)

                if temp_f_score < f_score[childCell]:
                    aPath[childCell] = currCell
                    g_score[childCell] = temp_g_score
                    f_score[childCell] = temp_g_score + h2(childCell, m._goal)
                    open.put((f_score[childCell], h2(
                        childCell, m._goal), childCell))

    fwdPath = {}
    cell = m._goal
    while cell != start:
        fwdPath[aPath[cell]] = cell
        cell = aPath[cell]
    return searchPath, aPath, fwdPath

In [5]:
m = maze(20, 30)

# Maze with multiple paths to reach goal
m.CreateMaze(loadMaze='Maze_1_mulptiplePaths.csv', theme="light")

searchPath1, aPath1, fwdPath1 = aStar1(m)
searchPath2, aPath2, fwdPath2 = aStar2(m)

textLabel(m, 'Manhattan Path Length', len(fwdPath1)+1)
textLabel(m, 'Euclidean Path Length', len(fwdPath2)+1)
textLabel(m, 'Manhattan Search Length', len(searchPath1)+1)
textLabel(m, 'Euclidean Search Length', len(searchPath2)+1)

a = agent(m, footprints=True, color=COLOR.cyan, filled=True)
b = agent(m, footprints=True, color=COLOR.red)

m.tracePath({a: fwdPath1}, delay=50)
m.tracePath({b: fwdPath2}, delay=50)

t1 = timeit(stmt='aStar1(m)', number=1000, globals=globals())
t2 = timeit(stmt='aStar2(m)', number=1000, globals=globals())

textLabel(m, 'Manhattan Time', t1)
textLabel(m, 'Euclidean Time', t2)

m.run()

In [6]:
f1, f2, f3 = 0, 0, 0
s1, s2, s3 = 0, 0, 0

for _ in range(100):
    myMaze = maze(20, 30)
    myMaze.CreateMaze(loopPercent=100)
    searchPath1, aPath1, fwdPath1 = aStar1(myMaze)
    searchPath2, aPath2, fwdPath2 = aStar2(myMaze)

    if len(fwdPath1) == len(fwdPath2):
        f1 += 1
    elif len(fwdPath1) < len(fwdPath2):
        f2 += 1
    else:
        f3 += 1

    if len(searchPath1) == len(searchPath2):
        s1 += 1
    elif len(searchPath1) < len(searchPath2):
        s2 += 1
    else:
        s3 += 1
        
print('Final Path Comparison Result')
print(f'\tBoth have same Final Path length for {f1} times.')
print(f'\tManhattan has lesser Final Path length for {f2} times.')
print(f'\tEuclidean has lesser Final Path length for {f3} times.')

print('\n--------------------------------------------\n')

print('Search Path Comparison Result')
print(f'\tBoth have same Search Path length for {s1} times.')
print(f'\tManhattan has lesser Search Path length for {s2} times.')
print(f'\tEuclidean has lesser Search Path length for {s3} times.')


Final Path Comparison Result
	Both have same Final Path length for 100 times.
	Manhattan has lesser Final Path length for 0 times.
	Euclidean has lesser Final Path length for 0 times.

--------------------------------------------

Search Path Comparison Result
	Both have same Search Path length for 0 times.
	Manhattan has lesser Search Path length for 100 times.
	Euclidean has lesser Search Path length for 0 times.
